In [203]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

In [264]:
train=pd.read_csv("../titanic/train.csv")
test=pd.read_csv("../titanic/test.csv")

In [205]:
train.head()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [287]:
#test의 Fare 빈값 채우기 
mean_fare =train[['Pclass', 'Fare']].groupby(['Pclass'], as_index=False).mean()
# test[pd.isnull(test['Fare'])]
test.loc[pd.isnull(test['Fare']),"Fare"]=mean_fare
test[pd.isnull(test['Fare'])]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Gender,Embarked_S,Embarked_C,Embarked_Q,FamilySize,Family,Family_S,Family_M,Family_L,Survived


In [266]:
train['Age_mean']=train['Age']
train['Age_mean'].isnull().sum() #177
train['Age_mean'].fillna(train['Age'].mean(),inplace=True)
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Age_mean         0
dtype: int64

In [267]:
test['Age_mean']=test['Age']
test['Age_mean'].fillna(test['Age'].mean(),inplace=True)

In [268]:
train['Sex'].value_counts()
train['Gender']=(train['Sex']=='female')
test['Gender']=(test['Sex']=='female')

In [269]:
train['Embarked_S']=train['Embarked']=='S'
train['Embarked_C']=train['Embarked']=='C'
train['Embarked_Q']=train['Embarked']=='Q'

In [270]:
test['Embarked_S']=test['Embarked']=='S'
test['Embarked_C']=test['Embarked']=='C'
test['Embarked_Q']=test['Embarked']=='Q'

In [271]:
train['FamilySize']=train['SibSp']+train['Parch']+1

In [272]:
train['Family']=train['FamilySize']
train.loc[train['FamilySize']==1, 'Family']='S'
train.loc[(train['FamilySize']>1) & 
          (train['FamilySize']<5), 'Family']='M'
train.loc[train['FamilySize']>=5, 'Family']='L'

In [273]:
train['Family_S']=train['Family']=='S'
train['Family_M']=train['Family']=='M'
train['Family_L']=train['Family']=='L'

In [274]:
test['FamilySize']=test['SibSp']+test['Parch']+1
test['Family']=test['FamilySize']
test.loc[test['FamilySize']==1, 'Family']='S'
test.loc[(test['FamilySize']>1) & 
          (test['FamilySize']<5), 'Family']='M'
test.loc[test['FamilySize']>=5, 'Family']='L'
test['Family_S']=test['Family']=='S'
test['Family_M']=test['Family']=='M'
test['Family_L']=test['Family']=='L'

In [275]:
train.loc[train['Fare']<=17,'Fare']=0
train.loc[(train['Fare']>17) & (train['Fare']<=30),'Fare']=1
train.loc[(train['Fare']>30) & (train['Fare']<=100),'Fare']=2
train.loc[(train['Fare']>100),'Fare']=3


In [276]:
test.loc[test['Fare']<=17,'Fare']=0
test.loc[(test['Fare']>17) & (test['Fare']<=30),'Fare']=1
test.loc[(test['Fare']>30) & (test['Fare']<=100),'Fare']=2
test.loc[(test['Fare']>100),'Fare']=3

In [277]:
train['Pclass'].value_counts()
train['Pclass']=train['Pclass'].astype('category')

In [278]:
fn=['Gender','Age_mean','Embarked_C', 'Embarked_Q','Family_S', 'Family_M', 'Family_L','Fare']
xTrain=train[fn]
xTrain.head()
yLable =train['Survived']

In [279]:
#모델 객체 생성
model=DecisionTreeClassifier(max_depth=4, random_state=2020)
#scikit-learn.org
model

DecisionTreeClassifier(max_depth=4, random_state=2020)

In [280]:
#모델링
model.fit(xTrain, yLable)

DecisionTreeClassifier(max_depth=4, random_state=2020)

In [281]:
from sklearn.model_selection import KFold, cross_val_score
#교차검증: 아직 본 적없는 데이터에 대해 성능을 예측
# 더 좋은 모델을 위한 파라미터 튜닝을 위해
Kfold=KFold(n_splits=10, shuffle= True, random_state=2020)
#전테 데이터를 10겹 으로 분리, 9겹 트레이닝, 1겹 테스트

In [282]:
score= cross_val_score(model, xTrain, yLable, cv=Kfold, n_jobs=-1, scoring='accuracy')
#n_jobs = 컴퓨터 코어사용 여부 -1은 전부 다 사용한다는 것

In [283]:
print(score)
round(np.mean(score)*100,2)

[0.9        0.71910112 0.84269663 0.80898876 0.78651685 0.83146067
 0.84269663 0.85393258 0.83146067 0.83146067]


82.48

In [284]:
xTest= test[fn]
xTest

,Gender,Age_mean,Embarked_C,Embarked_Q,Family_S,Family_M,Family_L,Fare
0,False,34.50000,False,True,True,False,False,0.0
1,True,47.00000,False,False,False,True,False,0.0
2,False,62.00000,False,True,True,False,False,0.0
3,False,27.00000,False,False,True,False,False,0.0
4,True,22.00000,False,False,False,True,False,0.0
...,...,...,...,...,...,...,...,...
413,False,30.27259,False,False,True,False,False,0.0
414,True,39.00000,True,False,True,False,False,3.0
415,False,38.50000,False,False,True,False,False,0.0
416,False,30.27259,False,False,True,False,False,0.0


In [285]:
prediction = model.predict(xTest)
prediction.shape
test['Survived']=prediction

In [286]:
res=test[['PassengerId', 'Survived']]
res.to_csv("titanic_res5.csv", index=False)

In [289]:
from sklearn.ensemble import RandomForestClassifier

In [298]:
#랜덤 포레스트 
clf = RandomForestClassifier(n_estimators=13)
scoring = 'accuracy'
score = cross_val_score(clf, xTrain, yLable, cv=Kfold, n_jobs=1, scoring=scoring)
print(score)

[0.77777778 0.68539326 0.7752809  0.78651685 0.75280899 0.7752809
 0.84269663 0.84269663 0.75280899 0.80898876]


In [299]:
clf.fit(xTrain, yLable)

RandomForestClassifier(n_estimators=13)

In [300]:
prediction = clf.predict(xTest)
prediction.shape
test['Survived']=prediction